### Creating 1 row per patient by aggregating and also calculating the relevant information

In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 500)   # to display 500 columns
pd.set_option('display.max_rows', 500) # to display 500 rows

import time
from datetime import datetime

In [2]:
import psycopg2

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database =  'cms_claims'  #'cms_medicare_claims'  

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

### Summarized history of a patient per row incl cost 

#### Getting list of patients who are common in 2008 and 2009

In [3]:
datetime.now()

datetime.datetime(2020, 5, 11, 22, 1, 45, 617327)

In [4]:

query = '''
   SELECT b2009."DESYNPUF_ID", b2008."BENE_BIRTH_DT"
   , 2009 - EXTRACT(YEAR FROM b2008."BENE_BIRTH_DT") as "AGE"
   
   FROM 
                  beneficiary2008 as b2008
       INNER JOIN beneficiary2009 as b2009
   On
       b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID"
       
   '''

common2008_2009DF = pd.read_sql(query , con=conn)

print(common2008_2009DF.shape)
common2008_2009DF.head(2)


(458328, 3)


,DESYNPUF_ID,BENE_BIRTH_DT,AGE
0,00048EF1F4791C68,1929-06-01,80.0
1,00052705243EA128,1934-05-01,75.0


#### 2008 and 2009 combined info per patient

In [5]:
query = """ 
        SELECT 
            b2008."DESYNPUF_ID",
            b2008."BENE_BIRTH_DT", 
            b2009."BENE_DEATH_DT", 
            b2008."BENE_SEX_IDENT_CD",
            b2008."BENE_RACE_CD", 
            b2008."BENE_ESRD_IND" as "END_STAGE_RENAL2008", 
            b2009."BENE_ESRD_IND" as "END_STAGE_RENAL2009", 
            b2008."SP_ALZHDMTA" as "SP_ALZHDMTA2008",
            b2009."SP_ALZHDMTA" as "SP_ALZHDMTA2009",
            b2008."SP_CHF" as "SP_CHF2008", 
            b2009."SP_CHF" as "SP_CHF2009", 
            b2008."SP_CHRNKIDN" as "SP_CHRNKIDN2008",
            b2009."SP_CHRNKIDN" as "SP_CHRNKIDN2009",
            b2008."SP_CNCR" as "SP_CNCR2008", 
            b2009."SP_CNCR" as "SP_CNCR2009", 
            b2008."SP_COPD" as "SP_COPD2008",
            b2009."SP_COPD" as "SP_COPD2009", 
            b2008."SP_DEPRESSN" as "SP_DEPRESSN2008",
            b2009."SP_DEPRESSN" as "SP_DEPRESSN2009",
            b2008."SP_DIABETES" as "SP_DIABETES2008",
            b2009."SP_DIABETES" as "SP_DIABETES2009",
            b2008."SP_ISCHMCHT" as "SP_ISCHMCHT2008", 
            b2009."SP_ISCHMCHT" as "SP_ISCHMCHT2009", 
            b2008."SP_OSTEOPRS" as "SP_OSTEOPRS2008",
            b2009."SP_OSTEOPRS" as "SP_OSTEOPRS2009",
            b2008."SP_RA_OA" as "SP_RA_OA2008", 
            b2009."SP_RA_OA" as "SP_RA_OA2009", 
            b2008."SP_STRKETIA" as "SP_STRKETIA2008",
            (22- (b2008."SP_ALZHDMTA"+ b2008."SP_CHF"+ b2008."SP_CHRNKIDN"+ b2008."SP_CNCR"+ b2008."SP_COPD"+
                    b2008."SP_DEPRESSN"+b2008."SP_DIABETES"+b2008."SP_ISCHMCHT"+ b2008."SP_OSTEOPRS"+ 
                    b2008."SP_RA_OA"+ b2008."SP_STRKETIA")) as "TOTAL_DIAGNOSIS2008",
            (22- (b2009."SP_ALZHDMTA"+ b2009."SP_CHF"+ b2009."SP_CHRNKIDN"+ b2009."SP_CNCR"+ b2009."SP_COPD"+
                    b2009."SP_DEPRESSN"+b2009."SP_DIABETES"+b2009."SP_ISCHMCHT"+ b2009."SP_OSTEOPRS"+ 
                    b2009."SP_RA_OA"+ b2009."SP_STRKETIA")) as "TOTAL_DIAGNOSIS2009",

           (b2008."MEDREIMB_IP" + b2008."BENRES_IP" + b2008."PPPYMT_IP" +  b2008."MEDREIMB_OP" + b2008."BENRES_OP" +
               b2008."PPPYMT_OP" + b2008."MEDREIMB_CAR" + b2008."BENRES_CAR" + b2008."PPPYMT_CAR" ) 
                                                                                           as "TOTAL_PAYMENT2008" ,
           (b2009."MEDREIMB_IP" + b2009."BENRES_IP" + b2009."PPPYMT_IP" +  b2009."MEDREIMB_OP" + b2009."BENRES_OP" +
               b2009."PPPYMT_OP" + b2009."MEDREIMB_CAR" + b2009."BENRES_CAR" + b2009."PPPYMT_CAR" ) 
                                                                                           as "TOTAL_PAYMENT2009",
        
        (b2009."MEDREIMB_IP" + b2009."BENRES_IP" + b2009."PPPYMT_IP" +  b2009."MEDREIMB_OP" + b2009."BENRES_OP" +
               b2009."PPPYMT_OP" + b2009."MEDREIMB_CAR" + b2009."BENRES_CAR" + b2009."PPPYMT_CAR" ) - 
        (b2008."MEDREIMB_IP" + b2008."BENRES_IP" + b2008."PPPYMT_IP" +  b2008."MEDREIMB_OP" + b2008."BENRES_OP" +
               b2008."PPPYMT_OP" + b2008."MEDREIMB_CAR" + b2008."BENRES_CAR" + b2008."PPPYMT_CAR" )
                                                         as "CHANGE_IN_PAYMEMT",
                                                                                               
                                                                            
        (22 - (b2009."SP_ALZHDMTA"+ b2009."SP_CHF"+ b2009."SP_CHRNKIDN"+ b2009."SP_CNCR"+ b2009."SP_COPD"+
                    b2009."SP_DEPRESSN"+b2009."SP_DIABETES"+b2009."SP_ISCHMCHT"+ b2009."SP_OSTEOPRS"+ 
                    b2009."SP_RA_OA"+ b2009."SP_STRKETIA")) - 
        (22-(b2008."SP_ALZHDMTA"+ b2008."SP_CHF"+ b2008."SP_CHRNKIDN"+ b2008."SP_CNCR"+ b2008."SP_COPD"+
                    b2008."SP_DEPRESSN"+b2008."SP_DIABETES"+b2008."SP_ISCHMCHT"+ b2008."SP_OSTEOPRS"+ 
                    b2008."SP_RA_OA"+ b2008."SP_STRKETIA")) as "CHANGE_IN_DIANOSIS"
                    
         , CASE WHEN b2009."BENE_DEATH_DT" IS NULL THEN 0 ELSE 1 END as "DEAD"
         , 2009 - EXTRACT(YEAR FROM b2008."BENE_BIRTH_DT") as "AGE"
                    
        FROM 
            beneficiary2008 as b2008
            INNER JOIN beneficiary2009 as b2009
        ON 
            b2008."DESYNPUF_ID" = b2009."DESYNPUF_ID"
        WHERE 
            b2008."DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
    
        ORDER BY b2009."DESYNPUF_ID"
        """

bene2008_2009DF = pd.read_sql(query , con=conn)
print(bene2008_2009DF.shape)
bene2008_2009DF.head(2)  

(458328, 36)


,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,END_STAGE_RENAL2008,END_STAGE_RENAL2009,SP_ALZHDMTA2008,SP_ALZHDMTA2009,SP_CHF2008,SP_CHF2009,SP_CHRNKIDN2008,SP_CHRNKIDN2009,SP_CNCR2008,SP_CNCR2009,SP_COPD2008,SP_COPD2009,SP_DEPRESSN2008,SP_DEPRESSN2009,SP_DIABETES2008,SP_DIABETES2009,SP_ISCHMCHT2008,SP_ISCHMCHT2009,SP_OSTEOPRS2008,SP_OSTEOPRS2009,SP_RA_OA2008,SP_RA_OA2009,SP_STRKETIA2008,TOTAL_DIAGNOSIS2008,TOTAL_DIAGNOSIS2009,TOTAL_PAYMENT2008,TOTAL_PAYMENT2009,CHANGE_IN_PAYMEMT,CHANGE_IN_DIANOSIS,DEAD,AGE
0,00000B48BCF4AD29,1923-09-01,NaT,2,5,0,0,1,1,1,1,1,2,1,2,2,2,1,1,1,1,1,1,1,1,2,2,1,9,6,93452.0,8750.0,-84702.0,-3,0,86.0
1,0000141F2FECE9BC,1974-04-01,NaT,2,1,0,0,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,0,1,0.0,5308.0,5308.0,1,0,35.0


#### Inpatient Claims

#### Adding flags for readmssions within 7 days, 30 days, 90 days

In [6]:
datetime.now()

datetime.datetime(2020, 5, 11, 22, 2, 0, 363722)

In [7]:
# Sum adds by row, if you want to add by columns then use + sign

query = """ 
SELECT 

    "DESYNPUF_ID"
    , "CLM_FROM_DT"
    , "CLM_THRU_DT"
        
        FROM 
            inpatient_claims
            where "DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
         
         order by "DESYNPUF_ID"
        
        """

inpt_claimsDF_temp = pd.read_sql(query , con=conn)
print(inpt_claimsDF_temp.shape)
inpt_claimsDF_temp.head(2)

# Find time difference between admissions - for each and then may be average - 
# not all pts wil have so may be have a flag for readmssion

(265422, 3)


,DESYNPUF_ID,CLM_FROM_DT,CLM_THRU_DT
0,00000B48BCF4AD29,2008-08-05,2008-08-20
1,00000B48BCF4AD29,2008-07-07,2008-07-18


##### Adding flag for readmssions within 7 days, 30 days, 90 days

In [8]:
#inpt_claimsDF_temp.sort_values(['DESYNPUF_ID','CLM_FROM_DT'])
datetime.now()

In [9]:
# 7 days readmission

inpt_claimsDF_temp['READMIT7']=inpt_claimsDF_temp.groupby('DESYNPUF_ID').apply(lambda x : (x['CLM_FROM_DT'].\
shift(-1)-x['CLM_THRU_DT']).dt.days.le(7).astype(int)).reset_index(0).drop('DESYNPUF_ID',1)

print(inpt_claimsDF_temp.shape)
#inpt_claimsDF_temp.head(20)

(265422, 4)


In [ ]:
datetime.now()

In [10]:
# 30 days readmission

inpt_claimsDF_temp['READMIT30']=inpt_claimsDF_temp.groupby('DESYNPUF_ID').apply(lambda x : (x['CLM_FROM_DT'].\
shift(-1)-x['CLM_THRU_DT']).dt.days.le(30).astype(int)).reset_index(0).drop('DESYNPUF_ID',1)

print(inpt_claimsDF_temp.shape)
#inpt_claimsDF_temp.head(20)

(265422, 5)


In [ ]:
datetime.now()

In [11]:
# 60 days readmission

inpt_claimsDF_temp['READMIT60']=inpt_claimsDF_temp.groupby('DESYNPUF_ID').apply(lambda x : (x['CLM_FROM_DT'].\
shift(-1)-x['CLM_THRU_DT']).dt.days.le(60).astype(int)).reset_index(0).drop('DESYNPUF_ID',1)

print(inpt_claimsDF_temp.shape)
#inpt_claimsDF_temp.head(20)

(265422, 6)


In [ ]:
datetime.now()

In [12]:
# 90 days readmission

inpt_claimsDF_temp['READMIT90']=inpt_claimsDF_temp.groupby('DESYNPUF_ID').apply(lambda x : (x['CLM_FROM_DT'].\
shift(-1)-x['CLM_THRU_DT']).dt.days.le(90).astype(int)).reset_index(0).drop('DESYNPUF_ID',1)

print(inpt_claimsDF_temp.shape)
#inpt_claimsDF_temp.head(20)

(265422, 7)


In [13]:
datetime.now()

#Readmit30 = inpt_claimsDF_temp.groupby(['DESYNPUF_ID'])['Readmit30'].sum().reset_index()
#print(Readmit30.shape)
#Readmit30.head(6)

In [14]:
# Now sum readmits for each patient

Readmit7  = inpt_claimsDF_temp.groupby(['DESYNPUF_ID'])['READMIT7'].sum().reset_index()
Readmit30 = inpt_claimsDF_temp.groupby(['DESYNPUF_ID'])['READMIT30'].sum().reset_index()
Readmit60 = inpt_claimsDF_temp.groupby(['DESYNPUF_ID'])['READMIT60'].sum().reset_index()
Readmit90 = inpt_claimsDF_temp.groupby(['DESYNPUF_ID'])['READMIT90'].sum().reset_index()
#Readmit60

In [15]:
datetime.now()

datetime.datetime(2020, 5, 12, 9, 19, 1, 774637)

In [16]:
# Sum adds by row, if you want to add by columns then use + sign

query = """ 
SELECT 

"DESYNPUF_ID"

, sum ("CLM_THRU_DT" - "CLM_FROM_DT") as "LOS"
, count("CLM_ID") as "NUM_INPT_ADM"
, sum (
        "CLM_PMT_AMT" + "NCH_BENE_IP_DDCTBL_AMT" + "NCH_PRMRY_PYR_CLM_PD_AMT" + "NCH_BENE_PTA_COINSRNC_LBLTY_AM" +
        "CLM_PASS_THRU_PER_DIEM_AMT" + "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM"
        ) as "TOTAL_INPT_COST"
,  ( 
        sum(CASE WHEN "ICD9_DGNS_CD_1" IS NOT NULL THEN 1 ELSE 0 END)
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_2" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_3" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_4" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_5" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_6" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_7" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_8" IS NOT NULL THEN 1 ELSE 0 END) 
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_9" IS NOT NULL THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_10" IS NOT NULL THEN 1 ELSE 0 END) 
        
    ) as "TOTAL_INPT_DIAGNOSIS"
  
, (
        sum(CASE WHEN "ICD9_PRCDR_CD_1"  IS NOT NULL THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN "ICD9_PRCDR_CD_2"  IS NOT NULL THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN "ICD9_PRCDR_CD_3"  IS NOT NULL THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN "ICD9_PRCDR_CD_4"  IS NOT NULL THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN "ICD9_PRCDR_CD_5"  IS NOT NULL THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN "ICD9_PRCDR_CD_6"  IS NOT NULL THEN 1 ELSE 0 END) 
        
    ) as "TOTAL_INPT_PROCS"
  
  
        FROM 
            inpatient_claims
            where "DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
         
        GROUP BY "DESYNPUF_ID" 
        ORDER BY "DESYNPUF_ID"
        
        """

inpt_claimsDF = pd.read_sql(query , con=conn)
print(inpt_claimsDF.shape)
inpt_claimsDF.head(2)

# Find time difference between admissions - for each and then may be average - 
# not all pts wil have so may be have a flag for readmssion

(150481, 6)


,DESYNPUF_ID,LOS,NUM_INPT_ADM,TOTAL_INPT_COST,TOTAL_INPT_DIAGNOSIS,TOTAL_INPT_PROCS
0,00000B48BCF4AD29,28 days,3,84072.0,27,11
1,0000141F2FECE9BC,2 days,1,5108.0,9,1


In [17]:
#inpt_claimsDF.DESYNPUF_ID.nunique()

In [18]:
datetime.now()

datetime.datetime(2020, 5, 12, 9, 19, 8, 212852)

#### Adding readmissions to the inpt_claims DataFrame

In [19]:
inpt_claimsDF = inpt_claimsDF.merge(Readmit7, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)


inpt_claimsDF = inpt_claimsDF.merge(Readmit30, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)


inpt_claimsDF = inpt_claimsDF.merge(Readmit60, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)


inpt_claimsDF = inpt_claimsDF.merge(Readmit90, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)

print(inpt_claimsDF.shape)

(150481, 10)


In [20]:
inpt_claimsDF.head(70)

,DESYNPUF_ID,LOS,NUM_INPT_ADM,TOTAL_INPT_COST,TOTAL_INPT_DIAGNOSIS,TOTAL_INPT_PROCS,READMIT7,READMIT30,READMIT60,READMIT90
0,00000B48BCF4AD29,28 days,3,84072.0,27,11,2,2,2,2
1,0000141F2FECE9BC,2 days,1,5108.0,9,1,0,0,0,0
2,0000525AB30E4DEF,17 days,2,14208.0,18,3,0,0,0,0
3,00009C897C3D8372,17 days,5,40096.0,45,2,3,3,3,3
4,0000C1386AE2C2A2,4 days,1,12170.0,9,2,0,0,0,0
5,0000EC65FBF94AB8,4 days,1,5068.0,9,0,0,0,0,0
6,00013D2EFD8E45D1,1 days,1,5100.0,9,0,0,0,0,0
7,00016F745862898F,16 days,4,56304.0,27,4,0,1,1,1
8,0002056B40CEE448,2 days,1,14068.0,9,1,0,0,0,0
9,00042C476CC26F05,2 days,1,13034.0,2,3,0,0,0,0


In [21]:
datetime.now()

datetime.datetime(2020, 5, 12, 9, 19, 8, 356635)

#### Outpatient Claims

In [22]:
query = """ 
SELECT opd."DESYNPUF_ID"
, count("CLM_ID") as "NUM_OPD_VISIT"
, sum (
        "CLM_PMT_AMT" + "NCH_PRMRY_PYR_CLM_PD_AMT" +  "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM" +
        "NCH_BENE_PTB_DDCTBL_AMT" + "NCH_BENE_PTB_COINSRNC_AMT"
        
    ) as "TOTAL_OPD_COST"

,  ( 
        sum(CASE WHEN "ICD9_DGNS_CD_1" not in ('None', 'Nan', 'NaN', 'NONE', '', ' ') THEN 1 ELSE 0 END)
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_2" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_3" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_4" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_5" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_6" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_7" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_8" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END) 
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_9" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END)  
        +  
        sum(CASE WHEN "ICD9_DGNS_CD_10" not in ('None', 'Nan', 'NaN', 'NONE') THEN 1 ELSE 0 END) 
        
    ) as "TOTAL_OPD_DIAGNOSIS"
  
, (
        sum(CASE WHEN cast("ICD9_PRCDR_CD_1" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("ICD9_PRCDR_CD_2" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("ICD9_PRCDR_CD_3" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("ICD9_PRCDR_CD_4" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("ICD9_PRCDR_CD_5" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("ICD9_PRCDR_CD_6" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        
    ) as "TOTAL_OPD_PROCS"
  
, (
        sum(CASE WHEN cast("HCPCS_CD_1" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("HCPCS_CD_2" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("HCPCS_CD_3" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("HCPCS_CD_4" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("HCPCS_CD_5" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        +
        sum(CASE WHEN cast("HCPCS_CD_6" as varchar(100)) not in ('None') THEN 1 ELSE 0 END) 
        
    ) as "TOTAL_OPD_HCPCS"
 
 
        FROM 
            outpatient_claims opd
        
        WHERE 
            opd."DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
         
        GROUP BY "DESYNPUF_ID"
        ORDER BY "DESYNPUF_ID"
        
        """

opd_claimsDF = pd.read_sql(query , con=conn)
print(opd_claimsDF.shape)
opd_claimsDF.head(2)

(338719, 6)


,DESYNPUF_ID,NUM_OPD_VISIT,TOTAL_OPD_COST,TOTAL_OPD_DIAGNOSIS,TOTAL_OPD_PROCS,TOTAL_OPD_HCPCS
0,00000B48BCF4AD29,10,3210.0,27,0,47
1,0000525AB30E4DEF,30,7870.0,99,0,67


In [ ]:
datetime.now()

#### No link with OPD and Inpatient claims tables to Carrier Clainms so Leaving Carrier Claims table

#### Prescription Events Table

In [23]:
query = """ 

SELECT 
    pde."DESYNPUF_ID"
    , sum("QTY_DSPNSD_NUM") as "TOTAL_QTY_DSPNSD_NUM"
    , sum("PTNT_PAY_AMT") as "PTNT_PAY_RX_AMT"
    , sum("TOT_RX_CST_AMT") as "TOT_RX_CST_AMT"
        
FROM 
    prescription_drug_events as pde

WHERE 
    pde."DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
     
        
    GROUP BY "DESYNPUF_ID"
    ORDER BY "DESYNPUF_ID"
        """

pres_eventsDF = pd.read_sql(query , con=conn)
print(pres_eventsDF.shape)
pres_eventsDF.head(5)

(395256, 4)


,DESYNPUF_ID,TOTAL_QTY_DSPNSD_NUM,PTNT_PAY_RX_AMT,TOT_RX_CST_AMT
0,00000B48BCF4AD29,210.0,10.0,200.0
1,0000141F2FECE9BC,9930.0,1740.0,13490.0
2,0000525AB30E4DEF,5380.0,1020.0,5390.0
3,00009C897C3D8372,3040.0,660.0,4380.0
4,0000C1386AE2C2A2,3060.0,950.0,4400.0


#### Concatenating all the Diagnosis and Procedure codes as string from Inpatient Claims Table

In [24]:
query = """ 
SELECT "DESYNPUF_ID"
  , CONCAT(' ', "ICD9_DGNS_CD_1", ' ', "ICD9_DGNS_CD_2", ' ', "ICD9_DGNS_CD_3", ' ', "ICD9_DGNS_CD_4", ' ', 
                               "ICD9_DGNS_CD_5", ' ', "ICD9_DGNS_CD_6", ' ', "ICD9_DGNS_CD_7", ' ', "ICD9_DGNS_CD_8", ' ', 
                              "ICD9_DGNS_CD_9", ' ', "ICD9_DGNS_CD_10") as "INPT_DIAGS" 
                              
  , CONCAT(' ', "ICD9_PRCDR_CD_1", ' ', "ICD9_PRCDR_CD_2", ' ', "ICD9_PRCDR_CD_3", ' ', "ICD9_PRCDR_CD_5", ' ', 
                               "ICD9_PRCDR_CD_5", ' ', "ICD9_PRCDR_CD_6") as "INPT_PROCS" 
 
FROM 
    inpatient_claims 
       
 WHERE  "DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
    
         
ORDER BY "DESYNPUF_ID" 
"""

inpt_diag_procDF = pd.read_sql(query , con=conn)
print(inpt_diag_procDF.shape)
inpt_diag_procDF.head(5)

(265422, 3)


,DESYNPUF_ID,INPT_DIAGS,INPT_PROCS
0,00000B48BCF4AD29,4241 2760 7802 2763 42821 2762 41410 3970 5119,3521 9971 5601 V451 V451 41071
1,00000B48BCF4AD29,3970 2800 9971 41410 V4581 2875 5121 4242 42731,3615 53081 42820 78321 78321 2749
2,00000B48BCF4AD29,42823 2859 5119 56400 4263 V4582 3051 25060 4...,
3,0000141F2FECE9BC,5789 79029 5363 5990 3569 2449 2851 311 2869,4516
4,0000525AB30E4DEF,5579 V4611 5849 99591 42731 4019 78550 53550 ...,3898 5579


In [25]:
inpt_diagDF = inpt_diag_procDF[["DESYNPUF_ID", "INPT_DIAGS"]].copy()
inpt_procDF = inpt_diag_procDF[["DESYNPUF_ID", "INPT_PROCS"]].copy()
inpt_diagDF.shape, inpt_procDF.shape

((265422, 2), (265422, 2))

In [26]:
inpt_diagDF = inpt_diagDF.groupby(['DESYNPUF_ID'])['INPT_DIAGS'].apply(lambda x: ''.join(x)).reset_index()
print(inpt_diagDF.shape)
inpt_diagDF.head(2)

(150481, 2)


,DESYNPUF_ID,INPT_DIAGS
0,00000B48BCF4AD29,4241 2760 7802 2763 42821 2762 41410 3970 511...
1,0000141F2FECE9BC,5789 79029 5363 5990 3569 2449 2851 311 2869


In [27]:
inpt_procDF = inpt_procDF.groupby(['DESYNPUF_ID'])['INPT_PROCS'].apply(lambda x: ''.join(x)).reset_index()
print(inpt_procDF.shape)
inpt_procDF.head(2)

(150481, 2)


,DESYNPUF_ID,INPT_PROCS
0,00000B48BCF4AD29,3521 9971 5601 V451 V451 41071 3615 53081 428...
1,0000141F2FECE9BC,4516


In [ ]:
datetime.now()

#### Concatenating all the Diagnosis, Procedure and HCPCS codes as string from Outpatient Claims Table

In [28]:
query = """ 

SELECT "DESYNPUF_ID"
  , CONCAT(' ', "ICD9_DGNS_CD_1", ' ', "ICD9_DGNS_CD_2", ' ', "ICD9_DGNS_CD_3", ' ', "ICD9_DGNS_CD_4", ' ', 
                               "ICD9_DGNS_CD_5", ' ', "ICD9_DGNS_CD_6", ' ', "ICD9_DGNS_CD_7", ' ', "ICD9_DGNS_CD_8", ' ', 
                              "ICD9_DGNS_CD_9", ' ', "ICD9_DGNS_CD_10") as "OPD_DIAGS" 
                              
  , CONCAT(' ', "ICD9_PRCDR_CD_1", ' ', "ICD9_PRCDR_CD_2", ' ', "ICD9_PRCDR_CD_3", ' ', "ICD9_PRCDR_CD_5", ' ', 
                               "ICD9_PRCDR_CD_5", ' ', "ICD9_PRCDR_CD_6") as "OPD_PROCS" 
 
  , CONCAT(' ', "HCPCS_CD_1", ' ', "HCPCS_CD_2", ' ', "HCPCS_CD_3", ' ', "HCPCS_CD_4", ' ',  "HCPCS_CD_5", ' ', 
           "HCPCS_CD_6", ' ', "HCPCS_CD_7", ' ', "HCPCS_CD_8",' ', "HCPCS_CD_9",' ', "HCPCS_CD_10",' ', 
           "HCPCS_CD_11", ' ', "HCPCS_CD_12",' ', "HCPCS_CD_13") as "OPD_HCPCS" 
    
FROM 
    outpatient_claims 
       
 WHERE  "DESYNPUF_ID" in (SELECT 
                                        b2009."DESYNPUF_ID"
                                    FROM 
                                              beneficiary2008 as b2008
                                    INNER JOIN beneficiary2009 as b2009
                                    On  b2009."DESYNPUF_ID" = b2008."DESYNPUF_ID")  
    
         
ORDER BY "DESYNPUF_ID" 
            
            """

opd_diag_procDF = pd.read_sql(query , con=conn)
print(opd_diag_procDF.shape)
opd_diag_procDF.head(2)

(3157412, 4)


,DESYNPUF_ID,OPD_DIAGS,OPD_PROCS,OPD_HCPCS
0,00000B48BCF4AD29,72999 72981,,93971
1,00000B48BCF4AD29,2721 V1250 2330,,G0103 80076 80053 80053 85610 87086 82248 825...


In [29]:
opd_diagDF = opd_diag_procDF[["DESYNPUF_ID", "OPD_DIAGS"]].copy()
opd_procDF = opd_diag_procDF[["DESYNPUF_ID", "OPD_PROCS"]].copy()
opd_hspcsDF = opd_diag_procDF[["DESYNPUF_ID", "OPD_HCPCS"]].copy()
opd_diagDF.shape, opd_procDF.shape, opd_hspcsDF.shape

((3157412, 2), (3157412, 2), (3157412, 2))

In [30]:
opd_diagDF = opd_diagDF.groupby(['DESYNPUF_ID'])['OPD_DIAGS'].apply(lambda x: ''.join(x)).reset_index()
print(opd_diagDF.shape)
opd_diagDF.head(2)

(338719, 2)


,DESYNPUF_ID,OPD_DIAGS
0,00000B48BCF4AD29,72999 72981 2721 V1250 2330 79...
1,0000525AB30E4DEF,32731 78079 5990 53081 OTHER 2...


In [ ]:
datetime.now()

In [31]:
opd_procDF = opd_procDF.groupby(['DESYNPUF_ID'])['OPD_PROCS'].apply(lambda x: ''.join(x)).reset_index()
print(opd_procDF.shape)
opd_procDF.head(2)

(338719, 2)


,DESYNPUF_ID,OPD_PROCS
0,00000B48BCF4AD29,...
1,0000525AB30E4DEF,...


In [ ]:
datetime.now()

In [32]:
opd_hspcsDF = opd_hspcsDF.groupby(['DESYNPUF_ID'])['OPD_HCPCS'].apply(lambda x: ''.join(x)).reset_index()
print(opd_hspcsDF.shape)
opd_hspcsDF.head(2)

(338719, 2)


,DESYNPUF_ID,OPD_HCPCS
0,00000B48BCF4AD29,93971 G0103 80076 80053 80053 856...
1,0000525AB30E4DEF,92557 A0436 80053 A95...


In [ ]:
datetime.now()

### Merging DataFrames by patient ID

In [33]:
bene2008_2009DF = bene2008_2009DF.merge(inpt_claimsDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 45)


In [34]:
bene2008_2009DF = bene2008_2009DF.merge(opd_claimsDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 50)


In [35]:
bene2008_2009DF = bene2008_2009DF.merge(pres_eventsDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 53)


In [36]:
bene2008_2009DF = bene2008_2009DF.merge(inpt_diagDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 54)


In [37]:
bene2008_2009DF = bene2008_2009DF.merge(inpt_procDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 55)


In [38]:
bene2008_2009DF = bene2008_2009DF.merge(opd_diagDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 56)


In [39]:
bene2008_2009DF = bene2008_2009DF.merge(opd_procDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 57)


In [40]:
bene2008_2009DF = bene2008_2009DF.merge(opd_hspcsDF, on='DESYNPUF_ID', 
                                              how='left',  left_index=True, right_index=True)
print(bene2008_2009DF.shape)

(458328, 58)


In [41]:
bene2008_2009DF.head(5)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,END_STAGE_RENAL2008,END_STAGE_RENAL2009,SP_ALZHDMTA2008,SP_ALZHDMTA2009,SP_CHF2008,SP_CHF2009,SP_CHRNKIDN2008,SP_CHRNKIDN2009,SP_CNCR2008,SP_CNCR2009,SP_COPD2008,SP_COPD2009,SP_DEPRESSN2008,SP_DEPRESSN2009,SP_DIABETES2008,SP_DIABETES2009,SP_ISCHMCHT2008,SP_ISCHMCHT2009,SP_OSTEOPRS2008,SP_OSTEOPRS2009,SP_RA_OA2008,SP_RA_OA2009,SP_STRKETIA2008,TOTAL_DIAGNOSIS2008,TOTAL_DIAGNOSIS2009,TOTAL_PAYMENT2008,TOTAL_PAYMENT2009,CHANGE_IN_PAYMEMT,CHANGE_IN_DIANOSIS,DEAD,AGE,LOS,NUM_INPT_ADM,TOTAL_INPT_COST,TOTAL_INPT_DIAGNOSIS,TOTAL_INPT_PROCS,READMIT7,READMIT30,READMIT60,READMIT90,NUM_OPD_VISIT,TOTAL_OPD_COST,TOTAL_OPD_DIAGNOSIS,TOTAL_OPD_PROCS,TOTAL_OPD_HCPCS,TOTAL_QTY_DSPNSD_NUM,PTNT_PAY_RX_AMT,TOT_RX_CST_AMT,INPT_DIAGS,INPT_PROCS,OPD_DIAGS,OPD_PROCS,OPD_HCPCS
0,00000B48BCF4AD29,1923-09-01,NaT,2,5,0,0,1,1,1,1,1,2,1,2,2,2,1,1,1,1,1,1,1,1,2,2,1,9,6,93452.0,8750.0,-84702.0,-3,0,86.0,28 days,3.0,84072.0,27.0,11.0,2.0,2.0,2.0,2.0,10.0,3210.0,27.0,0.0,47.0,210.0,10.0,200.0,4241 2760 7802 2763 42821 2762 41410 3970 511...,3521 9971 5601 V451 V451 41071 3615 53081 428...,72999 72981 2721 V1250 2330 79...,...,93971 G0103 80076 80053 80053 856...
1,0000141F2FECE9BC,1974-04-01,NaT,2,1,0,0,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,0,1,0.0,5308.0,5308.0,1,0,35.0,2 days,1.0,5108.0,9.0,1.0,0.0,0.0,0.0,0.0,30.0,7870.0,99.0,0.0,67.0,9930.0,1740.0,13490.0,5789 79029 5363 5990 3569 2449 2851 311 2869,4516,32731 78079 5990 53081 OTHER 2...,...,92557 A0436 80053 A95...
2,0000525AB30E4DEF,1920-10-01,NaT,2,1,0,0,1,1,1,1,1,1,2,2,1,1,2,2,1,1,1,1,2,1,2,2,1,7,8,22628.0,14730.0,-7898.0,1,0,89.0,17 days,2.0,14208.0,18.0,3.0,0.0,0.0,0.0,0.0,13.0,3110.0,33.0,0.0,37.0,5380.0,1020.0,5390.0,5579 V4611 5849 99591 42731 4019 78550 53550 ...,3898 5579 9915,5951 74190 25000 2724 V570 71536 496 ...,...,81003 97112 97110 97542 97110 97110 97110 971...
3,00009C897C3D8372,1932-01-01,NaT,1,1,Y,0,1,2,1,1,1,2,1,2,1,1,1,2,1,1,1,1,2,2,1,1,2,9,5,43736.0,3590.0,-40146.0,-4,0,77.0,17 days,5.0,40096.0,45.0,2.0,3.0,3.0,3.0,3.0,3.0,320.0,8.0,0.0,11.0,3040.0,660.0,4380.0,5070 485 4941 2724 49121 2639 4739 4019 5852 ...,8877 53081,5291 78079 V1269 2724 78060 4293 ...,,93005 92597 84100 J0690 P9035 82947 87101 8...
4,0000B27E77EE1987,1955-02-01,NaT,1,1,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,0,0.0,0.0,0.0,0,0,54.0,4 days,1.0,12170.0,9.0,2.0,0.0,0.0,0.0,0.0,9.0,1780.0,14.0,0.0,17.0,3060.0,950.0,4400.0,2859 5856 53540 51881 5859 79029 78791 0414 4...,4525 40390,V7799 V5869 V8281 2723 ...,...,G0202 77052 G0202 77052 ...


In [ ]:
datetime.now()

### Wrting this to a table in PostGres Database

In [42]:
# connect using sqlalchemy
import sqlalchemy

engine = sqlalchemy.engine.create_engine(conn_str)

In [43]:
pd.read_sql("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public'", con=engine)

,tablename
0,icd9_diagonsis
1,icd9_procedures
2,hcpcs
3,beneficiary2008
4,beneficiary2009
5,beneficiary2010
6,inpatient_claims
7,outpatient_claims
8,prescription_drug_events
9,final2008_2009


In [44]:
datetime.now()

datetime.datetime(2020, 5, 12, 9, 24, 0, 546603)

In [45]:
# Takes around 100 mins for 1 folder data - too 5 mins for 4 folders

bene2008_2009DF.to_sql(name = 'final2008_2009v1', con = engine, if_exists = 'replace', index = False)

/Users/abhi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  method=method,


In [46]:
datetime.now()

datetime.datetime(2020, 5, 12, 9, 28, 18, 175136)

In [47]:
conn.commit()   #engine.commit()

In [48]:
pd.read_sql("SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public'", con=engine)

,tablename
0,icd9_diagonsis
1,icd9_procedures
2,hcpcs
3,beneficiary2008
4,beneficiary2009
5,beneficiary2010
6,inpatient_claims
7,outpatient_claims
8,prescription_drug_events
9,final2008_2009
